In [2]:
# author: Kim Iheon, Yee Yechan
# version: 1

from scipy.special import expit as sigmoid


# 노드를 생성하고 저장
class Node:
    
    # 노드에 저장할 값을 초기화
    def __init__(self):
        
        # 노드에 들어온 입력값의 총합
        self.value_inf = 0
        
        # 입력값의 총합에 활성화 함수를 취한 값
        self.value_fin = 0
        
        # 입력값에 대한 오차의 변화량
        self.dE_dx = 0
        
        # 출력값에 대한 오차의 변화량
        self.dE_dy = 0
        
        # 노드로 들어오는 시냅스의 개수
        self.num_sys = 0
        
        # 작동한 시냅스의 개수
        self.count_sys = 0
        
        return
    
    # 노드에 값을 입력하는 함수
    # param {Float} delta_value 입력할 값
    def input_value(self, delta_value):
        
        # 기존의 입력 값에 변화량을 더함
        self.value_inf += delta_value
        
        # 작동한 시냅스의 개수를 늘림
        self.count_sys += 1
        
        return self.value_inf
    
    # 입력값에 활성화 함수를 취하는 함수
    def activation(self):
        
        # 입력값의 총합에 시그모이드를 취함
        self.value_fin = sigmoid(self.value_inf)
        
        return self.value_fin


# 신경망을 만들고 학습
class Network:
    
    # 유향 비순환 그래프 구조의 신경망 생성
    # param {Dictionary} network {시냅스: 가중치}의 신경망 딕셔너리
    # param {Float} 신경망의 학습률
    # param {Int} num_node 신경망의 초기 노드 수
    # param {Int} num_in 입력 노드의 개수
    # param {Int} num_out 출력 노드의 개수
    def __init__(self, network, learn_rate, num_node, num_in, num_out):
        self.network = network
        self.node_list = [Node() for i in range(num_node)]
        self.learn_rate = learn_rate
        self.num_node = num_node
        self.num_in = num_in
        self.num_out = num_out
        self.dE_dw = {}
        
        # 각 노드로 들어가는 시냅스의 개수 계산
        for sys in network:
            self.node_list[sys[1]].num_sys += 1
            
        return
    
    # 모든 노드의 시냅스 카운트를 초기화
    def restart(self):
        num_node = self.num_node
        
        for i in range(num_node)
            self.node_list[i].count_sys = 0
        
        return
        
    # 출력 값을 다음 노드에 입력하는 함수
    # param {Int} node 출력하는 노드의 번호
    def out_value(self, node):
        net = self.network
        
        for sys in net:
            if sys[0] == node:
                
                # 값을 받는 노드
                in_node = self.node_list[sys[1]]
                
                # 값을 주는 노드
                out_node = self.node_list[sys[0]]
                
                if out_node.value_inf == 0:
                
                    # 출력된 값과 가중치의 곱
                    in_value = out_node.value_fin * net[sys]
                    in_node.input_value(in_value)
        
        return
    
    # 순전파후 결과를 리턴하는 함수
    # param {List} input_list 입력노드에 입력할 값
    # return {List} out_value 출력노드가 출력할 값
    def calculate(self, input_list):
        net = self.network
        node_list = self.node_list
        num_node = self.num_node
        num_in = self.num_in
        num_out = self.num_out
        count_inf = [0]
        count_fin = [1]
        
        # 입력 노드에 값들을 입력함
        for i in range(num_in):
            self.node_list[i].value_inf = input_list[i]
        
        while count_inf != count_fin:

            # 계산 전 각 노드의 count 값
            count_inf = [node.count_sys for node in self.node_list]

            # 값이 다 입력된 노드를 작동시킴
            for node in self.node_list:
                if node.num_sys <= node.count_sys:
                    node.activation()

            for i in range(num_node):
                
                # 출력노드가 아니면 값을 다음노드에 넘겨줌
                if i < (num_node - num_out):
                    in_node = self.node_list[i]
                    if in_node.value_fin != 0:
                        self.out_value(i)

            # 계산 후 각 노드의 count 값
            count_fin = [node.count_sys for node in self.node_list]
        
        # 출력 노드 번호
        out_list = range(num_node - num_out, num_node)
        out_value = [self.node_list[o].value_fin for o in out_list]
        
        return out_value
    
    # 한 노드에 대해 역전파 연산을 수행
    # param {Int} node 연산을 수행할 노드
    def back(self, node):
        
        # {시냅스: 가중치} 딕셔너리
        net = self.network
        
        # node 에서 y에 대한 오차의 미분
        dE_dy = self.node_list[node].dE_dy
        
        # node 의 입력값의 총합
        x = self.node_list[node].value_inf
        
        # node 에서 입력값에 대한 출력값의 미분
        dy_dx = sigmoid(x) * (1 - sigmoid(x))
        
        # node 에서 입력값에 대한 오차의 미분을 체인룰로 구함
        dE_dx = dy_dx * dE_dy
        
        for sys in net:
            if sys[1] == node:
                
                # node 와 연결되어 있는 노드의 출력값
                y = self.node_list[sys[0]].value_fin
                
                # 시냅스의 가중치에 대한 오차의 미분
                self.dE_dw[sys] = y * dE_dx
                
                # 연결된 노드의 dE_dy를 계산
                self.node_list[sys[0]].dE_dy += net[sys] * dE_dx
        
        return